In [2]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, SecPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"usuarios regionales: {clients:,.0f} clientes")

#data
curr = SecApi()
curr.request('affected_detail',SecPayload())
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print('usuarios sin servicio :',f'{affected:,.0f}',' clientes',f'{affected/clients:.2%}')

res:DataFrame = curr_df.groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS'].sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=1) )
res

usuarios regionales: 780,563 clientes
usuarios sin servicio : 1,297  clientes 0.17%


,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
4,Villa Alemana,CHILQUINTA,2024-08-20,577,1 days 08:43:53.072288
5,Viña del Mar,CHILQUINTA,2024-02-19,513,184 days 08:43:53.072288
0,Cartagena,CHILQUINTA,2024-08-19,100,2 days 08:43:53.072288
1,Quilpue,CHILQUINTA,2024-02-19,83,184 days 08:43:53.072288
6,Viña del Mar,CHILQUINTA,2024-08-19,22,2 days 08:43:53.072288
2,Quilpue,CHILQUINTA,2024-08-20,1,1 days 08:43:53.072288
3,Valparaiso,CGE,2024-08-20,1,1 days 08:43:53.073288


In [6]:
res_by_city:DataFrame = curr_df.groupby(['NOMBRE_COMUNA'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res_by_city

,NOMBRE_COMUNA,CLIENTES_AFECTADOS
3,Villa Alemana,577
4,Viña del Mar,535
0,Cartagena,100
1,Quilpue,84
2,Valparaiso,1
